# Clustering Crypto

In [32]:
# Initial imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler =StandardScaler ()
mm = MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [33]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [34]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [35]:
# Alternatively, use the provided csv file:
file_path = pd.read_csv ("C:/Users/pvolc/rice_fintech/AWS-Lex/Instructions/Starter_Files/Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.DataFrame (file_path, index=None) 
crypto_df.tail()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
1251,PUNK,SteamPunk,PoS,False,PoS,NaN,40000000


### Data Preprocessing

In [36]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = pd.DataFrame (crypto_df,columns = ['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'])
crypto_df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1247,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
1251,SteamPunk,PoS,False,PoS,NaN,40000000


In [37]:
# Keep only cryptocurrencies that are trading
crypto_df.iloc[:, -1]
crypto_df = crypto_df.loc [crypto_df["IsTrading"] ==True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [38]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [39]:
# Keep only cryptocurrencies with a working algorithm
crypto_df.iloc[crypto_df["Algorithm"] == 0, 2:] = np.nan
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,1.0,PoW/PoS,4.199995e+01,42
1,365Coin,X11,1.0,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,1.0,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,1.0,PoW,NaN,611000
4,808,SHA-256,1.0,PoW/PoS,0.000000e+00,0


In [40]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(["IsTrading"], axis = 1)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [41]:
len (crypto_df)

1144

In [42]:
# Find null values
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [43]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna (how="any", axis=0)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [45]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc [crypto_df["TotalCoinsMined"] > 0]
crypto_df.dropna()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [61]:
crypto_clean = crypto_df.dropna()

In [62]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name = pd.DataFrame(file_path, columns = ["CoinName"])
coin_name = coin_name.set_index ("CoinName")
coin_name.head()

""
CoinName
42 Coin
365Coin
404Coin
SixEleven
808


In [49]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(["CoinName"], axis = 1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [50]:
# Create dummy variables for text features
crypto_df = pd.get_dummies (crypto_df, columns = ["Algorithm", "ProofType"])
crypto_df.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
# Standardize data
data_scaler = StandardScaler()

In [52]:
# Fitting the scaler
data_scaler.fit (crypto_df)

StandardScaler()

In [53]:
# Transforming the data
crypto_df = data_scaler.transform (crypto_df)

### Reducing Dimensions Using PCA

In [54]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [55]:
# Get two principal components for the iris data.
crypto_df = pca.fit_transform(crypto_df)

In [56]:
# Create a DataFrame with the principal components data
crypto_df = pd.DataFrame (data = crypto_df, columns=["PC 1", "PC 2", "PC 3"])
crypto_df.head()

,PC 1,PC 2,PC 3
0,-0.337766,1.047356,-0.516617
1,-0.321096,1.047314,-0.516875
2,2.307687,1.608367,-0.588617
3,-0.147831,-1.290161,0.158763
4,-0.150164,-2.027721,0.348677


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [57]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
pcs_df = {"k": k, "inertia": inertia}
pcs_df = pd.DataFrame(pcs_df)
pcs_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k= 7, inertia = 27.8

In [60]:
# Initialize the K-Means model with k = 7
model = KMeans(n_clusters=7, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.DataFrame (data = crypto_clean, columns=["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "PC 1", "PC 2", "PC 3", "CoinName", "Class"])
clustered_df.tail()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
1238,SHA-256,DPoS,2.000000e+09,2000000000,NaN,NaN,NaN,ZEPHYR,NaN
1242,Scrypt,PoW/PoS,1.493105e+07,250000000,NaN,NaN,NaN,Gapcoin,NaN
1245,CryptoNight,PoW,9.802226e+08,1400222610,NaN,NaN,NaN,Beldex,NaN
1246,Equihash,PoW,7.296538e+06,21000000,NaN,NaN,NaN,Horizen,NaN
1247,Scrypt,PoS,1.283270e+05,1000000,NaN,NaN,NaN,BitcoinPlus,NaN


### Visualizing Results

#### 3D-Scatter with Clusters

In [63]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=700,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

ValueError: nan is not in list

#### Table of Tradable Cryptocurrencies

In [92]:
# Table with tradable cryptos
df_cn = pd.DataFrame(file_path, columns = ["CoinName", "TotalCoinsMined", "TotalCoinSupply"]) 
df_cn.tail()

,CoinName,TotalCoinsMined,TotalCoinSupply
1247,BitcoinPlus,1.283270e+05,1000000
1248,DivotyCoin,2.149121e+07,100000000
1249,Giotto Coin,NaN,233100000
1250,OpenSourceCoin,NaN,21000000
1251,SteamPunk,NaN,40000000


In [93]:
# Print the total number of tradable cryptocurrencies
df_cn.dropna()

,CoinName,TotalCoinsMined,TotalCoinSupply
0,42 Coin,4.199995e+01,42
2,404Coin,1.055185e+09,532000000
4,808,0.000000e+00,0
5,EliteCoin,2.927942e+10,314159265359
7,Bitcoin,1.792718e+07,21000000
...,...,...,...
1242,Gapcoin,1.493105e+07,250000000
1245,Beldex,9.802226e+08,1400222610
1246,Horizen,7.296538e+06,21000000
1247,BitcoinPlus,1.283270e+05,1000000


In [94]:
df_cn = df_notnull.loc [df_notnull["TotalCoinsMined"] > 0]
df_cn.dropna()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
10,Dash,X11,PoW/PoS,9.031294e+06,22000000
16,BitcoinDark,SHA-256,PoW/PoS,1.288862e+06,22000000
...,...,...,...,...,...
1193,OKCash,SHA-256,PoW/PoS,7.467606e+07,105000000
1196,WhiteCoin,Scrypt,PoW/PoS,2.520056e+08,300000000
1198,FriendshipCoin,NeoScrypt,PoW/PoS,1.120385e+06,60168145
1202,Triangles Coin,X13,PoW/PoS,1.407778e+05,120000


#### Scatter Plot with Tradable Cryptocurrencies

In [114]:
# Scale data to create the scatter plot
#df_cn_scaled = scaler.fit_transform(df_cn[["TotalCoinsMined","TotalCoinSupply"]])

In [117]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
